# ASL Image Re-orientation Notebook

This is a notebook that takes an existing folder of ASL (American Sign Language) data (that already has a folder sub-structure to contain images of each letter seperately) and produces a new folder WITHIN THE EXISTING FOLDER that contains each image correctly orientated. The motivation for this project was due to the problem that, when importing images taken on different devices into PowerAI Vision the images would sometimes appear rotated therein. The reason for this rotation lies with the fact that different imaging devices (phones/Ipads etc) store the image differently depending on the orientation of the device when the picture was taken (e.g. in portrait or landscape). This rotation behaviour is apparently exclusive to Apple devices.

So first let's import some libraries:

In [1]:
from PIL import Image
from PIL import ExifTags
import os, sys

We now need to specify some inputs. These are described below.

input_dir : This is the name of the folder that contains our original images. Please note that this folder should already contain within itself 25 separate folders, one each for each letter of the alphabet (within which are located the image files themselves) and excluding the letter Z since we do not wish to identify it (this is because it is a motion for which the beginning and endpoints are not exclusively Z-like). It ought to commence with two backslashes.

ouput_dir : This is the name of the output folder to be produced. output_dir will be located within input_dir once the code has been successfully run. Please note we don't need to include any backslashes here.

full_path_to_input_dir : Self-explanatory. Note a forward slash is also needed on the end (and at the start!) and the path includes the input directory too.

dir : This should be the full path to input_dir. Hence please note that we must be located WITHIN input_dir before we execute any of the subsequent code.

Oh and in case you are wondering why we have to declare the path to the input directory twice, there is a discrepency in format between the two.

In [ ]:
input_dir = "\\ASL_Images_raw_21st_March"
output_dir = "resized_Images_21st_March"
full_path_to_input_dir = "/Users/PeterFox/Documents/ASL/ASL_Images_raw_21st_March/"
dir = os.getcwd() 

Quick check that we are currently within input_dir:

In [249]:
os.getcwd()

'C:\\Users\\PeterFox\\Documents\\ASL\\ASL_Images_raw_21st_March'

Good! Now let's do some juicy coding :) It will be useful to have a list of all the (capitalised) alphabet characters, for when we wish to re-create our folder structure. For this we can simply do the following (excluding Z):

In [238]:
alphabet = map(chr, range(65, 90))

Now let's make a function called create_output_folder whose purpose in life is, like all well-written functions, self-evident.

In [239]:
def create_output_folder(letter):
    if not os.path.exists(os.path.join(dir,output_dir,letter)):
        os.mkdir(os.path.join(dir,output_dir,letter))

The following function takes an input image, performs a re-orientation if it is required and then saves the image (rotated or not) into its letter sub-folder. Please note the image MUST currently have extension .jpg, .JPG, .jpeg or .JPEG. Although I should think the code could be amended to take other image types such as png if so required.

In [245]:
def reorientateImage(infile, output_dir, letter):
     outfile = os.path.splitext(infile)[0] + "_reorientated"
     extension = os.path.splitext(infile)[1]

     if (extension == ".jpg") or (extension == ".JPG") or (extension == ".jpeg") or (extension == ".JPEG"):

        if infile != outfile:

            try :
                im = Image.open(infile)
                exif = im._getexif()
                exif = dict(exif.items())
                
                if 271 in exif:
                    if exif[271] == 'Apple':
                        if exif[274] == 3:
                            im=im.rotate(180, expand=True)
                        elif exif[274] == 8:
                            im = im.rotate(90, expand=True)
                        elif exif[274] == 6:
                            im = im.rotate(270, expand=True)
                        else:
                            pass;

                absolute_name_with_path = full_path_to_input_dir + output_dir + "/" + letter + 'correct_orientation' + outfile + ".JPEG"
                im.save(absolute_name_with_path,"JPEG")
            except IOError:
                print ("cannot reduce image for ", infile)
                

And let's use both these functions to create our new directory structure and do our image re-orientation.

In [242]:
def change_folder_and_perform_image_reorientation_if_required():
    for letter in alphabet:
        #if not os.path.exists(os.path.join(dir,output_dir,letter)):
        create_output_folder(letter)
        os.chdir(dir + "\\" + letter)
        for file in os.listdir("."):
            reorientateImage(file, output_dir, letter + "/")

Now all that remains for us to do is call our function, right?! Almost - but first we should use our create_output_folder function to produce our output directory:

In [ ]:
create_output_folder("")

Now we are ready to call our uber-function...

In [234]:
change_folder_and_perform_image_reorientation_if_required()

And that's it! This code could be easily amended to do some image re-sizing if required.